<a href="https://colab.research.google.com/github/cloudmadeofcandy/temporal_skeleton/blob/master/Test_Multi_agent_Workflow_with_LangGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Create tools and RAG

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [ ]:
!pip install -q chromadb langchain-chroma langchain-community langchain-experimental langchain_google_genai langchain_classic pypdf lxml ddgs pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.6/209.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.6/326.6 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 51.8 MB/s eta 0:00:

## Init a Chroma database and Collection

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

chat_model = ChatGoogleGenerativeAI(model='gemini-2.5-flash', api_key=GOOGLE_API_KEY)
pro_chat_model = ChatGoogleGenerativeAI(model='gemini-2.5-pro', api_key=GOOGLE_API_KEY)
embedding_model = GoogleGenerativeAIEmbeddings(model='gemini-embedding-001', google_api_key=GOOGLE_API_KEY)

In [ ]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="quantq2_test_database",
    embedding_function=embedding_model,
    persist_directory="./drive/MyDrive/chroma_langchain_db",
)

## Import pdfs, split and persist into vectordb

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
file_path = "./drive/MyDrive/Deep-Learning-Specialization-notes.pdf"
loader = PyPDFLoader(file_path, mode="page")

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=1000*15/100,
    length_function=len,
    is_separator_regex=False,
)

In [ ]:
import pprint
docs = loader.load()
print(len(docs))
pprint.pp(docs[0].metadata)

100
{'producer': 'www.ilovepdf.com',
 'creator': 'PyPDF',
 'creationdate': '',
 'moddate': '2019-07-20T16:33:45+10:00',
 'source': './drive/MyDrive/Deep-Learning-Specialization-notes.pdf',
 'total_pages': 100,
 'page': 0,
 'page_label': '1'}


In [ ]:
metadata_key=['moddate', 'source', 'page']
chunks = {}
chunks = text_splitter.split_documents(docs)

In [ ]:
chunks[0]

Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2019-07-20T16:33:45+10:00', 'source': './drive/MyDrive/Deep-Learning-Specialization-notes.pdf', 'total_pages': 100, 'page': 0, 'page_label': '1'}, page_content='Course summary\nHere are the course summary as its given on the course link:\nIf you want to break into cutting-edge AI, this course will help you do so. Deep learning engineers are highly sought\nafter, and mastering deep learning will give you numerous new career opportunities. Deep learning is also a new\n"superpower" that will let you build AI systems that just weren\'t possible a few years ago.\nIn this course, you will learn the foundations of deep learning. When you finish this class, you will:\nUnderstand the major technology trends driving Deep Learning\nBe able to build, train and apply fully connected deep neural networks\nKnow how to implement efficient (vectorized) neural networks\nUnderstand the key parameters in

In [ ]:
# Synchronous ver:
# vector_id_list = vector_store.add_documents(chunks)

# Asynchronous ver:
vector_id_list = await vector_store.aadd_documents(chunks)

In [ ]:
vector_id_list

In [ ]:
a = await vector_store.asimilarity_search_with_relevance_scores("mastering deep learning will give you numerous new career opportunities")

In [ ]:
a

In [ ]:
chat_model.invoke("Hello World")

## TRY TO EXTRACT IMG

In [ ]:
import pymupdf
from bs4 import BeautifulSoup

In [ ]:
file_path = "./drive/MyDrive/Deep-Learning-Specialization-notes.pdf"
doc = pymupdf.open(file_path)

In [ ]:
from langchain_text_splitters.html import HTMLSemanticPreservingSplitter

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2")
    ]

semantic_preserving_text_splitter = HTMLSemanticPreservingSplitter(
    headers_to_split_on=headers_to_split_on,
    max_chunk_size=1000,
    preserve_links=True,
    preserve_images=True,
)

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=1000*15/100,
    length_function=len,
    is_separator_regex=False,
)

/tmp/ipython-input-1016297299.py:8: LangChainBetaWarning: The class `HTMLSemanticPreservingSplitter` is in beta. It is actively being worked on, so the API may change.
  semantic_preserving_text_splitter = HTMLSemanticPreservingSplitter(


In [ ]:
# CREATE A NEW COLLECTION:

vector_store_w_img = Chroma(
    collection_name="quantq2_test_w_img_database",
    embedding_function=embedding_model,
    persist_directory="./drive/MyDrive/chroma_langchain_db_w_img",
)

In [ ]:
# from pprint import pprint
# from bs4 import BeautifulSoup
# import uuid

# text_tags = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p']

# for page_index in range(len(doc)): # iterate over pdf pages
#     metadata = {
#         'page_number': page_index,
#         'file_path': file_path,
#     }
#     page = doc[page_index] # get the page
#     html_content = page.get_text(option='html')
#     soup = BeautifulSoup(html_content, "html.parser")
#     elements = soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'img', 'p'])
#     text = ""
#     for element in elements:
#         if element.name in text_tags:
#             text = text + " " + element.get_text()
#     pprint(text)
#     break

In [ ]:
from typing import List, Literal, Any
from pprint import pprint
from bs4 import BeautifulSoup
import uuid
from langchain_core.documents.base import Document

def consolidate_img_caption(
        siblings: List[Any] = [],
        order: Literal['forward', 'reverse'] = 'forward',
        char_limit: int = 1000
        ) -> str:

    ret = ""

    if order not in ['forward', 'reverse']:
        raise ValueError("The `order` must be either 'forward' or 'reverse")

    limit = 0
    text_stack = []

    for i in siblings:
        sibling_text = i.get_text()
        limit += len(sibling_text)

        if limit > char_limit:
            break
        else:
            text_stack.append(sibling_text)

    if order == 'reverse':
        text_stack.reverse()

    ret = " ".join(text_stack)

    return ret
    pass

In [ ]:
from typing import List, Literal, Any
from pprint import pprint
from bs4 import BeautifulSoup
import uuid
from langchain_core.documents.base import Document
from tqdm import tqdm
import re


text_tags = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p']
headers_to_split_on = [('h1', 'Heading 1'),
                       ('h2', 'Heading 2'),
                       ('h3', 'Heading 3'),
                       ('h4', 'Heading 4'),
                       ('h5', 'Heading 5'),
                       ('h6', 'Heading 6'),
                    #    ('p', 'Paragraph'),
                       ('img', 'Images')
                    ]

semantic_preserving_text_splitter = HTMLSemanticPreservingSplitter(
    headers_to_split_on=headers_to_split_on,
    max_chunk_size=1000,
    preserve_links=True,
    preserve_images=True,
)

doc_documents = []

for page_index in tqdm(range(len(doc))): # iterate over pdf pages

    page = doc[page_index] # get the page
    html_content = page.get_text(option='html')

    soup = BeautifulSoup(html_content, "html.parser")

    img_tags = []

    for i in soup.find_all('img'):
        image_notification_tag = soup.new_tag('p')
        image_notification_tag.string = str(uuid.uuid4())

        previous_siblings = i.find_previous_siblings()
        next_siblings = i.find_next_siblings()

        text_previous = consolidate_img_caption(previous_siblings, 'reverse', 250)
        text_next = consolidate_img_caption(next_siblings, 'forward', 250)

        img = i.get('src')
        img = img.replace("\n", "")
        img = re.sub(r'^data:image/\w+;base64,', '', img)

        i.replace_with(image_notification_tag)

        metadata = {
            'page_number': page_index,
            'file_path': file_path,
            'image': img
        }

        page_content = " ".join([text_previous, text_next])

        document = Document(
            page_content = page_content,
            metadata = metadata
        )

        img_tags.append(document)

    soup_documents = semantic_preserving_text_splitter.split_text(str(soup))
    soup_documents = soup_documents + img_tags

    for index, i in enumerate(soup_documents):
        if len(i.metadata) == 0:
            i.metadata = {
            'page_number': page_index,
            'file_path': file_path,
            'chunk_number': index
        }

    doc_documents += soup_documents


100%|██████████| 100/100 [00:04<00:00, 22.21it/s]


In [ ]:
# vector_id_list = await vector_store_w_img.aadd_documents(doc_documents)

In [ ]:
a = await vector_store_w_img.asimilarity_search_with_relevance_scores("How do we create a binary classifier with logistic regression")

In [ ]:
from langchain_classic.retrievers import MultiQueryRetriever
from langchain_core.prompts.prompt import PromptTemplate

retriever_w_img = vector_store_w_img.as_retriever()

n_question = 3

MULTI_QUERY_PROMPT_mqp = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate {n_question}
    different versions of the given user question to retrieve relevant documents from a vector
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search.
    Provide these alternative questions separated by newlines.
    Original question: {question}""",
    partial_variables =  {"n_question": n_question},
)

multi_query_retriever = MultiQueryRetriever.from_llm(
    llm=chat_model,
    retriever=retriever_w_img,
    prompt=MULTI_QUERY_PROMPT_mqp
)

In [ ]:
# Expand the search space by enhancing the input

from langchain_core.callbacks.manager import CallbackManagerForRetrieverRun

query = "How do we create a binary classifier with logistic regression"
query_rag_response = multi_query_retriever.invoke(query)


In [ ]:
query_rag_response

[Document(id='301b1107-e927-43f4-9633-fa379eed5a27', metadata={'file_path': './drive/MyDrive/Deep-Learning-Specialization-notes.pdf', 'image': 'iVBORw0KGgoAAAANSUhEUgAAAWgAAACrCAIAAADEjJEsAAAACXBIWXMAAA7EAAAOxAGVKw4bAAASH0lEQVR4nO3d/XMT950HcP0LneHqTDI3GnqZOJPr2O3QgeRmApfpJITWRikhZw4IqWsIrhkSxhMSLkdap6QObsEi116cqYvvksOYSShOih/kZ/OoYAw2GD8IjB8wfsSxLGMsY5xyH7PJervySvtd7bPer9kfLCFLX0T0zve9+9Wu4z4AACOH0QMAAOtBcAAAMwQHADBDcAAAMwQHADBDcAAAM+bgmBkernc4hJu/pibib9FjRL9Fz6NowABgPCUzjtHjx4UR0LJ69WwgEObx9Kf0GOGv0DMoHTAAGE9hVbmani4MgoH8/DAPpj8VPph+V9mLAoBJKAyO6d5eUfWge2Q+csrni2LAAGA85TtHRfOI7t27F3wY3S9/bgIAlhDVURXRnouA1yt6AN0j2hsSzcsBgElEFRwRcyFisgCAFUW7jqNv/36pwyWigy/0yChfCwBMItrgCF3WwS3QkLofAGxAhZWjosVd3MxCNBORs0gMAKxCnSXnomUdw0VFWLgBYGPqBEfoYg05SzwAwKJU+5KbaJYhnH2o9RIAYBJqfjtWdPAVCzcA7ErN4Jjy+UTBcbu5WcXnB7Cxvz1wbzo4UF014j1LPxs9onBUPh+HKDjUfXIA++Hy4uuvvx5ra207kHtqfUr9C8nntqXfu3eP7jR6dJIQHAB/p7i4eNeuN1eseIq2HTu2FxUVBYNBLV6Iy4vJgYGuT494t/yC8kK4DTU3UXaYdt6B4AD4Rnd397JlP0xKXvLrrJ+WlGbQ9v5e19q1yxITH29v

In [ ]:
chat_model.invoke("Hello?")

AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--7ef5ae7c-7404-40b0-9cfe-77261bf500fd-0', usage_metadata={'input_tokens': 3, 'output_tokens': 44, 'total_tokens': 47, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 35}})

In [ ]:
|import base64

with open("imageToSave.png", "wb") as fh:
    fh.write(base64.b64decode(full_text))

In [ ]:
import cv2
import matplotlib.pyplot as plt

# Path to your image file
image_path = "imageToSave.png"

# Read the image using OpenCV
img = cv2.imread(image_path)

# Check if the image was loaded successfully
if img is None:
    print(f"Error: Could not load image from {image_path}")
else:
    # Convert BGR to RGB for correct display with Matplotlib
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Display the image in the notebook
    plt.imshow(img_rgb)
    plt.axis('off')  # Turn off axis labels and ticks
    plt.title("")
    plt.show()

## DuckDuckGo Search



In [ ]:
ddgs_search = ddgs